In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import dill
from IPython.core import display as ICD  # to print multiple nice pandas tables
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import tqdm
from tqdm import tqdm_notebook, tnrange
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of datasets failed: Traceback (most recent call last):
  File "/home/vlejd/.virtualenvs/diplomka/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
KeyError: 'SENTEVAL_DATA_BASE'
]


In [14]:
import datasets
import classify
from simple_model import SimpleModel
from testing import test_simple_model

In [15]:
from collections import defaultdict
results = defaultdict(dict)
dump = defaultdict(dict)

In [16]:
def gradient_w(model, dataset, alpha=0.01, epochs=150):
    train_scores = []
    valid_scores = []
    test_scores = []
    model.fit(dataset.train_samples(), dataset.train_labels())
    if epochs is None:
        t = tnrange(100000)
    else:
        t = tnrange(epochs)
    for e in t:
        w = model.get_matrix_w()
        w -= alpha * model.dw(dataset.train_samples(), dataset.train_labels())
        model.save_matrix_w(w)
        model.fit(dataset.train_samples(), dataset.train_labels())

        train_score = model.score(dataset.train_samples(), dataset.train_labels())
        valid_score = model.score(dataset.valid_samples(), dataset.valid_labels())
        test_score = model.score(dataset.test_samples(), dataset.test_labels())
        train_scores.append(train_score)
        valid_scores.append(valid_score)
        test_scores.append(test_score)
        t.set_postfix(train_score=train_score, valid_score=valid_score, test_score=test_score)
        
        if epochs is None and e > 30:
            end_mean = np.mean(valid_scores[-10:])
            previos_mean = np.mean(valid_scores[-20:-10])
            t.set_postfix(train_score=train_score, valid_score=valid_score, test_score=test_score, previos=previos_mean, end=end_mean)
            if end_mean < previos_mean:
                break
    plt.plot(train_scores)
    plt.plot(valid_scores)
    plt.plot(test_scores)
    plt.legend(['train', 'valid', 'test'])
    plt.show()
    return train_scores, valid_scores, test_scores

In [17]:
def test_simple_model_with_gradient(
    model, dataset, gradient_iters=300, dims=300, alpha=0.01, tag=None, results=None, dump=None, with_models=False, folds=1):
    
    for i in dataset.reshufle(None, folds):
        train_ps, valid_ps, test_ps = gradient_w(model, dataset, alpha, gradient_iters)
        #train_p = model.score(dataset.train_samples(), dataset.train_labels())
        #test_p = model.score(dataset.test_samples(), dataset.test_labels())

        train_p = np.mean(train_ps[-10:])
        valid_p = np.mean(valid_ps[-10:])
        test_p = np.mean(test_ps[-10:])
        if results is not None:
            results[dataset.name()][('batch', tag, alpha, dims, 'train', i)] = train_p
            results[dataset.name()][('batch', tag, alpha, dims, 'valid', i)] = valid_p
            results[dataset.name()][('batch', tag, alpha, dims, 'test', i)] = test_p

        if dump is not None:
            dump[dataset.name()][('batch', tag, alpha, i)] = {
                'train': list(train_ps),
                'valid': list(valid_ps),
                'test': list(test_ps),
                'w': model.internal_w,
            }
            if with_models:
                dump[dataset.name()][('batch', tag, alpha, i)]['model']= model


        #raw_results[dataset.name()][('gradientw', tag, alpha)] = (train_ps, test_ps)
        print(dataset.name())
        print("Train precision", train_p)
        print("Valid precision", valid_p)
        print("Test precision", test_p)

In [18]:
start = True
start_on = ' '
dump_results = False

result_file_pattern = 'dumps_new/batch_results_{}.pickle'
dump_file_pattern = 'dumps_new/batch_dump_{}.pickle'

for dataset in datasets.ALL_DATASETS+ datasets.TREC_DATASETS:
    for scheme in SimpleModel.SCHEMES:
        for alpha in [0.1, 0.01, 0.001]:
            for dims in [200, 300, 400]:
                tag = '{}_{}_{}_{}'.format(dataset.name(), scheme, alpha, dims)
                if not start:
                    start = (tag == start_on)
                    continue
                print(dataset.name(), scheme, alpha, dims, tag)
                model = SimpleModel(classify.SkClassifier(), use_svd=True, weights=scheme, svd_dim=dims)
                test_simple_model_with_gradient(
                    model, dataset, alpha=alpha, dims=dims, tag=scheme, 
                    gradient_iters=None, results=results, dump=dump, with_models=False, folds=5)
                print(list(model.internal_w.items())[:10])
                results_file = result_file_pattern.format(tag)
                dumps_file = dump_file_pattern.format(tag)
                if dump_results:
                    pickle.dump(results, open(results_file, 'bw'))
                    pickle.dump(dump, open(dumps_file, 'bw'))
                    results = defaultdict(dict)
                    dump = defaultdict(dict)
                

CRDataset None 0.1 200 CRDataset_None_0.1_200


TypeError: test_simple_model_with_gradient() got an unexpected keyword argument 'fold'

In [19]:
pd.DataFrame(results)

""
